In [1]:
import cv2
import mediapipe as mp
import pyautogui
import tkinter as tk
from PIL import Image, ImageTk

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

tip_ids = [4, 8, 12, 16, 20]
keys_down = set()

root = tk.Tk()
root.title("Gesture Control")
root.geometry("700x600")

label = tk.Label(root, text="Gesture: ", font=("Arial", 20))
label.pack()

video_label = tk.Label(root)
video_label.pack()

cap = cv2.VideoCapture(0)
screen_w, screen_h = pyautogui.size()

def fingers_up(hand_landmarks, handedness):
    fingers = []
    lm = hand_landmarks.landmark

    # Thumb
    if handedness == "Right":
        fingers.append(1 if lm[tip_ids[0]].x < lm[tip_ids[0] - 1].x else 0)
    else:
        fingers.append(1 if lm[tip_ids[0]].x > lm[tip_ids[0] - 1].x else 0)

    for id in range(1, 5):
        fingers.append(1 if lm[tip_ids[id]].y < lm[tip_ids[id] - 2].y else 0)
    return fingers

def press_key(key):
    if key not in keys_down:
        pyautogui.keyDown(key)
        keys_down.add(key)

def release_key(key):
    if key in keys_down:
        pyautogui.keyUp(key)
        keys_down.remove(key)

def release_all_keys():
    for key in list(keys_down):
        pyautogui.keyUp(key)
        keys_down.remove(key)

def update_frame():
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    gesture_texts = []

    if result.multi_hand_landmarks and result.multi_handedness:
        for hand_landmark, hand_handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            mp_draw.draw_landmarks(img, hand_landmark, mp_hands.HAND_CONNECTIONS)

            hand_label = hand_handedness.classification[0].label
            finger_status = fingers_up(hand_landmark, hand_label)

            gesture_text = f"{hand_label} Hand: "

            if hand_label == "Right":
                # Mouse Control
                h, w, _ = img.shape
                cx = int(hand_landmark.landmark[9].x * screen_w)
                cy = int(hand_landmark.landmark[9].y * screen_h)

                if finger_status == [1, 1, 1, 1, 1]:  # Index only
                    gesture_text += "Move Mouse"
                    pyautogui.moveTo(cx, cy)
                elif finger_status == [0, 1, 1, 0, 0]:  # Index + Middle
                    gesture_text += "Left Click"
                    pyautogui.click()
                elif finger_status == [0, 1, 0, 0, 0]:  # Index + Middle
                    gesture_text += "Right Click"
                    pyautogui.click(button='right')
                elif sum(finger_status) == 0:  # Fist
                    gesture_text += "Stop Mouse"
                else:
                    gesture_text += "Idle"

            elif hand_label == "Left":
                # Keyboard Control
                if finger_status == [1, 0, 0, 0, 0]:
                    gesture_text += "Jump"
                    press_key('space')
                elif finger_status == [1, 1, 1, 1, 1]:
                    gesture_text += "Move Forward"
                    press_key('w')
                elif finger_status == [0, 0, 1, 0, 0]:
                    gesture_text += "Move Left"
                    press_key('a')
                elif finger_status == [0, 0, 0, 1, 0]:
                    gesture_text += "Move Right"
                    press_key('d')
                elif finger_status == [0, 0, 0, 0, 1]:
                    gesture_text += "Move Backward"
                    press_key('s')
                elif sum(finger_status) == 0:
                    gesture_text += "Stop Keys"
                    release_all_keys()
                else:
                    gesture_text += "Idle"
                    release_all_keys()

            gesture_texts.append(gesture_text)

    else:
        gesture_texts.append("No Hand")
        release_all_keys()

    label.config(text=" | ".join(gesture_texts))

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=img_pil)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    video_label.after(10, update_frame)

def on_close():
    release_all_keys()
    cap.release()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)
root.bind("<Escape>", lambda event: on_close())
update_frame()
root.mainloop()


## HILL CLIMBER

In [3]:
import cv2
import mediapipe as mp
import pyautogui
import tkinter as tk
from PIL import Image, ImageTk

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

tip_ids = [4, 8, 12, 16, 20]

root = tk.Tk()
root.title("Gesture Control")
root.geometry("700x600")

label = tk.Label(root, text="Gesture: ", font=("Arial", 20))
label.pack()

video_label = tk.Label(root)
video_label.pack()

cap = cv2.VideoCapture(0)
screen_w, screen_h = pyautogui.size()

def fingers_up(hand_landmarks, handedness):
    fingers = []
    lm = hand_landmarks.landmark

    # Thumb
    if handedness == "Right":
        fingers.append(1 if lm[tip_ids[0]].x < lm[tip_ids[0] - 1].x else 0)
    else:
        fingers.append(1 if lm[tip_ids[0]].x > lm[tip_ids[0] - 1].x else 0)

    for id in range(1, 5):
        fingers.append(1 if lm[tip_ids[id]].y < lm[tip_ids[id] - 2].y else 0)
    return fingers

def press_key(key):
    if key not in keys_down:
        pyautogui.keyDown(key)
        keys_down.add(key)

def release_key(key):
    if key in keys_down:
        pyautogui.keyUp(key)
        keys_down.remove(key)

def release_all_keys():  
    for key in list(keys_down):
        pyautogui.keyUp(key)
        keys_down.remove(key)

def update_frame():
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    gesture_texts = []

    if result.multi_hand_landmarks and result.multi_handedness:
        for hand_landmark, hand_handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            mp_draw.draw_landmarks(img, hand_landmark, mp_hands.HAND_CONNECTIONS)

            hand_label = hand_handedness.classification[0].label
            finger_status = fingers_up(hand_landmark, hand_label)

            gesture_text = f"{hand_label} Hand: "

            if hand_label == "Right":
                # Mouse Control
                h, w, _ = img.shape
                cx = int(hand_landmark.landmark[9].x * screen_w)
                cy = int(hand_landmark.landmark[9].y * screen_h)

                if finger_status == [0, 0, 1, 0, 0]:  # Index only
                    gesture_text += "Move Mouse"
                    pyautogui.moveTo(cx, cy)
                elif finger_status == [0, 1, 1, 0, 0]:  # Index + Middle
                    gesture_text += "Left Click"
                    pyautogui.click()
                elif finger_status == [0, 1, 0, 0, 0]:  # Index + Middle
                    gesture_text += "Right Click"
                    pyautogui.click(button='right')
                elif finger_status == [1, 1, 1, 1, 1]:
                    gesture_text += "Move Right"
                    press_key('d')
                elif sum(finger_status) == 0:  # Fist
                    gesture_text += "Stop Mouse"
                else:
                    gesture_text += "Idle"

            elif hand_label == "Left":
                # Keyboard Control
                if finger_status == [1, 0, 0, 0, 0]:
                    gesture_text += "Jump"
                    press_key('space')
                elif finger_status == [1, 1, 1, 1, 1]:
                    gesture_text += "Move Forward"
                    press_key('a')
                elif finger_status == [1, 1, 1, 1, 1]:
                    gesture_text += "Move Left"
                    press_key('w')
                
                elif finger_status == [0, 0, 0, 0, 1]:
                    gesture_text += "Move Backward"
                    press_key('s')
                elif sum(finger_status) == 0:
                    gesture_text += "Stop Keys"
                    release_all_keys()
                else:
                    gesture_text += "Idle"
                    release_all_keys()

            gesture_texts.append(gesture_text)

    else:
        gesture_texts.append("No Hand")
        release_all_keys()

    label.config(text=" | ".join(gesture_texts))

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=img_pil)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    video_label.after(10, update_frame)

def on_close():
    release_all_keys()
    cap.release()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)
root.bind("<Escape>", lambda event: on_close())
update_frame()
root.mainloop()


NameError: name 'keys_down' is not defined